In [1]:
import numpy as np
import pandas as pd


In [2]:
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [3]:
column_list = ['学历年龄','年齢','勤続年数','所定内実労働時間数','超過実労働時間数','きまって支給する現金給与額(千円)',
               '所定内給与額(千円)','年間賞与その他特別給与額(千円)','労働者数(十人)','year']

In [4]:
df_list = []

In [5]:
for year in range(2009,2019):
    
        
    df = pd.read_excel(f'05-19data/05-19正社员/{year}.xls',sheet_name = ' E 製造業')
    if year == 2010:
        df = df.iloc[11:,8:17]
    else:
        df = df.iloc[13:,8:17]
    df['year'] = int(year)
    df.columns = column_list
    df_list.append(df)
    

In [6]:
data = pd.concat(df_list)

In [7]:
data['year'].value_counts()


2009    195
2010    195
2011    195
2012    195
2013    195
2014    195
2015    195
2016    195
2017    195
2018    195
Name: year, dtype: int64

In [8]:
data.reset_index(inplace=True,drop = 'index')

In [9]:
del_list = ['\u3000\u3000～１９歳','６０～６４歳','６５～６９歳','７０歳～']

In [10]:
drop_list = []

In [11]:
for item in del_list:
    drop_index = data[data['学历年龄'] == item].index
    drop_list.append(drop_index)

In [12]:
for item in drop_list:
    data.drop(index = item,inplace=True)

In [13]:
data.reset_index(inplace=True,drop = 'index')

In [14]:
data.head()

,学历年龄,年齢,勤続年数,所定内実労働時間数,超過実労働時間数,きまって支給する現金給与額(千円),所定内給与額(千円),年間賞与その他特別給与額(千円),労働者数(十人),year
0,男 女 計\n学 歴 計,40.8,14.5,162,11,325.8,300.8,1145.4,491523,2009
1,２０～２４歳,22.7,2.8,164,13,214.2,191.5,522,37795,2009
2,２５～２９歳,27.6,5.1,162,14,251.8,224.9,768.9,56957,2009
3,３０～３４歳,32.6,8.6,163,15,291.4,259.6,960.4,65214,2009
4,３５～３９歳,37.5,12.7,162,13,326.5,295.8,1141.8,75683,2009


In [15]:
data.iloc[36,0]

'大学・大学院卒'

In [16]:
def edu_trans(x):
    if x in ['男\u3000 女\u3000 計\n学\u3000 歴   計','\u3000\u3000 男\n学   歴   計','\u3000\u3000 女\n学   歴   計']:
        return 'delete'
    elif x =='中   学   卒':
        return 9
    elif x == '高   校   卒':
        return 12
    elif x == '高専・短大卒':
        return 14
    elif x == '大学・大学院卒':
        return 16

In [17]:
data['education'] = data['学历年龄'].map(edu_trans)

In [18]:
def sex_trans(x):
    if x in ['男\u3000 女\u3000 計\n学\u3000 歴   計']:
        return 'delete'
    elif x =='\u3000\u3000 男\n学   歴   計':
        return 1
    elif x == '\u3000\u3000 女\n学   歴   計':
        return 0

In [19]:
data['sex'] = data['学历年龄'].map(sex_trans)

In [20]:
data.fillna(method='ffill',inplace=True)

In [21]:
drop_list = []

In [22]:
drop_index = data[(data['sex'] == 'delete' )| (data['education']=='delete')].index

In [23]:
data.drop(index = drop_index,inplace=True)

In [24]:
data.reset_index(inplace=True,drop = 'index')

In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 720 entries, 0 to 719
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   学历年龄               720 non-null    object 
 1   年齢                 720 non-null    float64
 2   勤続年数               720 non-null    float64
 3   所定内実労働時間数          720 non-null    int64  
 4   超過実労働時間数           720 non-null    int64  
 5   きまって支給する現金給与額(千円)  720 non-null    float64
 6   所定内給与額(千円)         720 non-null    float64
 7   年間賞与その他特別給与額(千円)   720 non-null    float64
 8   労働者数(十人)           720 non-null    int64  
 9   year               720 non-null    int64  
 10  education          720 non-null    object 
 11  sex                720 non-null    object 
dtypes: float64(5), int64(4), object(3)
memory usage: 67.6+ KB


In [26]:
data.describe()

,年齢,勤続年数,所定内実労働時間数,超過実労働時間数,きまって支給する現金給与額(千円),所定内給与額(千円),年間賞与その他特別給与額(千円),労働者数(十人),year
count,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000
mean,40.265556,13.489722,165.315278,14.676389,305.701806,276.031667,888.397222,13150.812500,2013.500000
std,10.851923,7.591722,2.937796,6.381649,91.714232,88.053198,480.721019,30477.226884,2.874278
min,22.200000,1.200000,156.000000,2.000000,136.100000,120.500000,25.100000,153.000000,2009.000000
25%,32.475000,7.100000,164.000000,10.000000,229.550000,210.075000,556.850000,1422.500000,2011.000000
50%,41.300000,13.600000,165.000000,13.000000,292.250000,255.950000,805.050000,4062.000000,2013.500000
75%,47.600000,19.325000,167.000000,20.000000,371.450000,324.725000,1148.925000,15392.250000,2016.000000
max,58.200000,33.100000,186.000000,35.000000,554.800000,536.700000,2515.000000,239718.000000,2018.000000


In [27]:
data.to_excel('05-19data/washed_data.xlsx')